In [ ]:
# SmartSDLC Pro - Clean Python Script Version (no notebook metadata issues)

!pip install gradio transformers PyPDF2

import gradio as gr
import PyPDF2
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import json
import tempfile
import os
import datetime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 8.5 MB/s eta 0:00:00


In [ ]:
MODEL_ID = "ibm-granite/granite-3.3-2b-instruct"
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(MODEL_ID, torch_dtype=torch.float16)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

In [ ]:
SUPPORTED_LANGUAGES = ["Python", "Java", "JavaScript", "C++", "C", "C#", "Go", "TypeScript", "Swift", "Kotlin"]
DEFAULT_LANGUAGE = "Python"
TEST_FRAMEWORKS = {
    "Python": "pytest",
    "Java": "JUnit",
    "JavaScript": "Jest",
    "C++": "Google Test",
    "C": "Unity",
    "C#": "NUnit",
    "TypeScript": "Jest",
    "Swift": "XCTest",
    "Kotlin": "JUnit"
}

In [ ]:
@torch.inference_mode()
def query_granite(prompt: str, max_length=2048) -> str:
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_length=max_length)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

def process_requirements(pdf_file, language: str, progress=gr.Progress()):
    try:
        with tempfile.NamedTemporaryFile(delete=False) as tmp:
            if isinstance(pdf_file, bytes):
                tmp.write(pdf_file)
            else:
                with open(pdf_file.name, 'rb') as f:
                    tmp.write(f.read())
            tmp_path = tmp.name

        reader = PyPDF2.PdfReader(tmp_path)
        total_pages = len(reader.pages)
        results = []

        for i, page in enumerate(reader.pages):
            progress(i/total_pages, desc=f"Processing page {i+1}/{total_pages}")
            text = page.extract_text() or ""
            if text.strip():
                prompt = f"""Analyze requirements from page {i+1} for {language}:
{text}
Return JSON with:
- Requirements summary
- Complexity analysis
- Implementation priorities"""
                results.append(query_granite(prompt))

        os.unlink(tmp_path)

        if not results:
            return {"error": "No readable text found in PDF"}, "Error: No text found"

        combined = query_granite(f"Combine these analyses:\n{json.dumps(results)}")
        return {
            "status": "complete",
            "pages_processed": total_pages,
            "results": results,
            "combined_analysis": combined
        }, "Analysis Complete"

    except Exception as e:
        return {"error": str(e)}, f"Error: {str(e)}"
    finally:
        if 'tmp_path' in locals() and os.path.exists(tmp_path):
            os.unlink(tmp_path)

def generate_code(user_story: str, language: str):
    base_prompt = f"""Generate production-ready {language} code:
{user_story}

Include:
1. Error handling
2. Comments
3. Modern practices
4. Clean structure"""

    if language == "C":
        base_prompt += "\n5. Memory management\n6. Pointer safety\n7. Standard compliance (C99/C11)"
    return query_granite(base_prompt)

def debug_code(code: str, language: str):
    base_prompt = f"""Debug this {language} code and explain fixes:
{code}

Provide:
1. Fixed code
2. Bug explanations
3. Best practices"""

    if language == "C":
        base_prompt += "\n4. Memory leak analysis\n5. Pointer usage review\n6. Undefined behavior checks"
    return query_granite(base_prompt)

def create_tests(input_spec: str, language: str):
    framework = TEST_FRAMEWORKS.get(language, "standard")
    base_prompt = f"""Generate {framework} tests for {language}:
{input_spec}

Include:
1. Happy path
2. Edge cases"""

    if language == "C":
        base_prompt += "\n3. Memory allocation tests\n4. Pointer validity checks\n5. Boundary conditions"
    else:
        base_prompt += "\n3. Setup/teardown\n4. Clear names"
    return query_granite(base_prompt)


def document_code(code: str, language: str):
    base_prompt = f"""Document this {language} code:
{code}

Include:
1. Purpose
2. Inputs/Outputs
3. Examples"""

    if language == "C":
        base_prompt += "\n4. Memory management details\n5. Thread safety\n6. Compilation instructions"
    else:
        base_prompt += "\n4. Architecture"
    return query_granite(base_prompt)

def consult_ai(question: str, context: str = ""):
    prompt = f"""As a senior architect, answer:
{question}

Context: {context}

Provide:
1. Detailed explanation
2. Code examples
3. Best practices
4. Alternatives"""
    return query_granite(prompt)

In [ ]:
def save_feedback(feedback_text, rating):
    feedback_data = {
        "text": feedback_text,
        "rating": rating,
        "timestamp": str(datetime.datetime.now())
    }
    with open("feedback_log.json", "a") as f:
        f.write(json.dumps(feedback_data) + "\n")
    return "Thank you for your feedback!"

with gr.Blocks(title="SmartSDLC Pro", theme=gr.themes.Soft()) as app:
    gr.HTML("""<div style='text-align: center; padding: 20px; background: linear-gradient(135deg, #0062ff, #8a3ffc); color: white; border-radius: 8px; margin-bottom: 20px;'>
        <h1>SmartSDLC Pro – AI Development Suite</h1>
        <p>Complete SDLC automation with multi-language support</p>
    </div>""")


In [ ]:
language = gr.Dropdown(label="Programming Language", choices=SUPPORTED_LANGUAGES, value=DEFAULT_LANGUAGE)

    with gr.Tabs():
        with gr.Tab("📋 Requirements"):
            pdf_input = gr.File(label="Upload PDF", file_types=[".pdf"])
            analyze_btn = gr.Button("Analyze Requirements")
            status = gr.Textbox(label="Processing Status", interactive=False)
            req_output = gr.JSON(label="Analysis Results")

        with gr.Tab("💻 Code Generation"):
            story_input = gr.Textbox(label="User Story", lines=3)
            gen_btn = gr.Button("Generate Code")
            clear_code = gr.Button("Clear")
            code_output = gr.Code(label="Generated Code", interactive=True, language="python")

        with gr.Tab("🐞 Debug Code"):
            code_input = gr.Textbox(label="Your Code", lines=10)
            debug_btn = gr.Button("Debug Code")
            clear_debug = gr.Button("Clear")
            debug_output = gr.Code(label="Debug Results", interactive=True, language="python")

        with gr.Tab("🧪 Generate Tests"):
            test_input = gr.Textbox(label="Test Specifications", lines=5)
            test_btn = gr.Button("Generate Tests")
            clear_tests = gr.Button("Clear")
            test_output = gr.Code(label="Generated Tests", interactive=True, language="python")

        with gr.Tab("📝 Documentation"):
            doc_input = gr.Textbox(label="Code to Document", lines=10)
            doc_btn = gr.Button("Generate Docs")
            clear_docs = gr.Button("Clear")
            doc_output = gr.Textbox(label="Documentation", lines=12, interactive=True)

        with gr.Tab("💬 Ask AI Architect"):
            question = gr.Textbox(label="Your Question", lines=3)
            context = gr.Textbox(label="Context (Optional)", lines=2)
            ask_btn = gr.Button("Consult AI")
            clear_ai = gr.Button("Clear")
            advice = gr.Textbox(label="Professional Advice", lines=12, interactive=True)

        with gr.Tab("⭐ Feedback"):
            feedback_text = gr.Textbox(label="Your Feedback", lines=5)
            rating = gr.Radio(["⭐", "⭐⭐", "⭐⭐⭐", "⭐⭐⭐⭐", "⭐⭐⭐⭐⭐"], label="Your Rating", value="⭐⭐⭐")
            submit_btn = gr.Button("Submit Feedback")
            feedback_confirmation = gr.Textbox(label="Status", interactive=False)

    analyze_btn.click(process_requirements, [pdf_input, language], [req_output, status])
    gen_btn.click(generate_code, [story_input, language], code_output)
    clear_code.click(lambda: ("", ""), outputs=[story_input, code_output])
    debug_btn.click(debug_code, [code_input, language], debug_output)
    clear_debug.click(lambda: ("", ""), outputs=[code_input, debug_output])
    test_btn.click(create_tests, [test_input, language], test_output)
    clear_tests.click(lambda: ("", ""), outputs=[test_input, test_output])
    doc_btn.click(document_code, [doc_input, language], doc_output)
    clear_docs.click(lambda: ("", ""), outputs=[doc_input, doc_output])
    ask_btn.click(consult_ai, [question, context], advice)
    clear_ai.click(lambda: ("", "", ""), outputs=[question, context, advice])
    submit_btn.click(save_feedback, [feedback_text, rating], feedback_confirmation)
    language.change(lambda x: gr.Code.update(language=x.lower()), inputs=language, outputs=[code_output, debug_output, test_output])

    gr.HTML(f"""<div style='text-align: center; margin-top: 20px; padding: 12px; background: #f5f7fb; border-radius: 8px;'>
        <small>SmartSDLC Pro v3.1 | Running on {device.upper()} | Powered by IBM Granite</small>
    </div>""")

app.launch(share=True)